In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

## 1. outlier检验

In [9]:
def detect_outliers(data):
    """
    Detect outliers in numeric features of a DataFrame using the 3σ principle.
    
    """
    # Calculate the mean and standard deviation for each numeric feature
    numeric_features = data.select_dtypes(include=['number']).columns
    means = data[numeric_features].mean()
    stds = data[numeric_features].std()
    
    # Create a new column 'outlier_flag' to indicate whether there are outliers
    data['FLAG_OUTLIER'] = 0
    
    for feature in numeric_features:
        # Calculate the lower and upper limits based on the 3σ principle
        lower_limit = means[feature] - 3 * stds[feature]
        upper_limit = means[feature] + 3 * stds[feature]
        
        # Mark values outside the limits as outliers
        data.loc[(data[feature] < lower_limit) | (data[feature] > upper_limit), 'FLAG_OUTLIER'] = 1
    
    return data

## 2. 填充缺失值

In [10]:
#填充缺失值
def fillna_cata_data(train_data,test_data):
    train_data.fillna('XNA', inplace=True)
    test_data.fillna('XNA', inplace=True)
    return train_data,test_data

def fillna_number_data(data):
    data['exc_tf_1'] = data['EXT_SOURCE_1'].apply(lambda x: 1 if pd.isnull(x) else 0)
    data['exc_tf_2'] = data['EXT_SOURCE_2'].apply(lambda x: 1 if pd.isnull(x) else 0)
    data['exc_tf_3'] = data['EXT_SOURCE_3'].apply(lambda x: 1 if pd.isnull(x) else 0)
    means = data.mean()
    data.fillna(means, inplace=True)
    return data


## 3. 上采样和下采样

In [15]:
def oversampling(data):
    Y = data["TARGET"]
    X = data.loc[:, data.columns != "TARGET"]
    cata_col = X.select_dtypes(include=object).columns
    cata_col_indices = [X.columns.get_loc(col) for col in cata_col]
    sm = SMOTENC(random_state=1, categorical_features=cata_col_indices, k_neighbors=3)
    X_res, y_res = sm.fit_resample(X, Y)

    ## array->df
    X = pd.DataFrame(X_res)
    # get new index
    origin_ID = X_res.index.to_list()
    num_respl = X.shape[0]-len(origin_ID)
    resample_ID = [i for i in range(origin_ID[-1]+1,origin_ID[-1]+num_respl+1)]
    origin_ID.extend(resample_ID) #combine origin and resample id

    # combine X、y和index
    resampled_data = pd.concat([pd.DataFrame(X_res), pd.Series(y_res, name='TARGET')], axis=1)
    resampled_data = pd.concat([pd.Series(origin_ID, name="ID"),resampled_data],axis=1)
    resampled_data.set_index("ID", inplace=True)

    return resampled_data
    

def undersampling(data):
    # Get the target variable and features
    Y = data["TARGET"]
    X = data.drop(columns=["TARGET"])
    
    # Create a RandomUnderSampler object
    rus = RandomUnderSampler(random_state=1)
    X_res, y_res = rus.fit_resample(X, Y)
    
    # Create a new DataFrame to store the resampled data
    # Use a dictionary to build the DataFrame and ensure the indices are from the original dataset
    resampled_data = pd.concat([pd.DataFrame(X_res, columns=X.columns), pd.Series(y_res, name='TARGET')], axis=1)
    
    # Select the corresponding indices from the original dataset
    original_index = data.iloc[rus.sample_indices_].index
    
    # Set the indices of the resampled data to the original indices
    resampled_data.set_index(original_index, inplace=True)
    
    return resampled_data


## 4. encode

In [12]:
def encode_cata_data(train_data,test_data):
    
    # Initialize empty DataFrames to hold the encoded training and testing data
    encoder_train = pd.DataFrame(index = train_data.index)
    encoder_test = pd.DataFrame(index = test_data.index)
    
    # Initialize the encoders
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    label_encoder = LabelEncoder()
    
    for column in test_data.columns:
        unique_values = train_data[column].nunique()
        
        if unique_values == 2:  # For two-class features
            encoder_train[column] = label_encoder.fit_transform(train_data[column])
            encoder_test[column] = label_encoder.transform(test_data[column])
        else:  # For multi-class features
            # Fit the OneHotEncoder on the training data for the current column
            one_hot_encoder.fit(train_data[[column]])  
            
            ohe_encoded_train = one_hot_encoder.transform(train_data[[column]])  # Transform the training data using the fitted OneHotEncoder
            ohe_encoded_train = pd.DataFrame(ohe_encoded_train, columns=one_hot_encoder.get_feature_names_out([column]),index = train_data.index)
            encoder_train = pd.concat([encoder_train, ohe_encoded_train], axis=1)
            
            # Use the same fitted OneHotEncoder to transform the test data
            ohe_encoded_test = one_hot_encoder.transform(test_data[[column]])
            ohe_encoded_test = pd.DataFrame(ohe_encoded_test, columns=one_hot_encoder.get_feature_names_out([column]),index = test_data.index)
            encoder_test = pd.concat([encoder_test, ohe_encoded_test], axis=1)
            
    return encoder_train,encoder_test


## 5. correlation

In [13]:
def detect_multicollinearity(data, threshold=0.8):
    """
    Detect multicollinearity using Pearson correlation coefficient and plot a heatmap.
    
    """
    # Compute the correlation matrix
    corr_matrix = data.corr().abs()
    
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlations above the threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Plot the heatmap
    plt.figure(figsize=(25, 25))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()
    
    return corr_matrix

In [16]:
# 读取 CSV 文件
application_train = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_train.csv').set_index('SK_ID_CURR')
application_test = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_test.csv').set_index('SK_ID_CURR')

# 打印 DataFrame
application_train.head(5)

#1.detect outliers
application_train = detect_outliers(application_train)
application_test = detect_outliers(application_test)
'''这里不太懂
num_train_outliers = (train_data['FLAG_OUTLIER'] == 1).sum()
num_test_outliers = (test_data['FLAG_OUTLIER'] == 1).sum()
print(f"Number of train-outliers: {num_train_outliers}")
print(f"Number of teat-outliers: {num_test_outliers}")
'''

#2.fillna
cata_train, cata_test = fillna_cata_data(application_train.select_dtypes(include=['object']), application_test.select_dtypes(include=['object']))
number_train = fillna_number_data(application_train.select_dtypes(include=['number']))
number_test = fillna_number_data(application_test.select_dtypes(include=['number']))
train_data = pd.merge(cata_train,number_train, left_index=True, right_index=True)
test_data = pd.merge(cata_test,number_test, left_index=True, right_index=True)
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)

#3.采样
# OverSp_X, OverSp_y = oversampling(train_data)
# UnderSp_X, UnderSp_y = undersampling(train_data)
# print(f'Resampled dataset samples per class {Counter(OverSp_y)}')
# print(f'Resampled dataset samples per class {Counter(UnderSp_y)}')




In [17]:
testm = oversampling(train_data)
testm

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,FLAG_OUTLIER,exc_tf_1,exc_tf_2,exc_tf_3,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0.000000,0.000000,0.000000,0.000000,1.000000,1,0,0,0,1
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,1,0
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,...,0.007000,0.034362,0.267395,0.265474,1.899974,0,1,0,1,0
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,...,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565367,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,...,0.001402,0.006882,0.053551,0.053167,3.579426,1,1,0,0,1
565368,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,XNA,...,0.000000,0.752439,0.000000,0.000000,1.504878,1,1,0,0,1
565369,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,...,0.000000,0.000000,0.000000,2.300876,6.000000,1,1,0,0,1


In [18]:
testu = undersampling(train_data)
testu

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,FLAG_OUTLIER,exc_tf_1,exc_tf_2,exc_tf_3,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
365251,Revolving loans,F,N,Y,Unaccompanied,Pensioner,Higher education,Married,House / apartment,XNA,...,0.000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
116517,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,XNA,...,0.000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
178767,Cash loans,F,Y,Y,Unaccompanied,State servant,Incomplete higher,Married,House / apartment,Core staff,...,0.000,1.000000,0.000000,0.000000,8.000000,1,1,0,0,0
445020,Cash loans,F,N,N,Unaccompanied,Working,Incomplete higher,Married,House / apartment,Core staff,...,0.007,0.034362,0.267395,0.265474,1.899974,1,0,0,1,0
416457,Cash loans,F,N,N,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Sales staff,...,0.000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456186,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,...,0.000,0.000000,0.000000,0.000000,2.000000,1,0,0,0,1
456215,Cash loans,F,N,N,Unaccompanied,Working,Higher education,Civil marriage,House / apartment,Laborers,...,0.000,0.000000,1.000000,0.000000,6.000000,1,0,0,0,1
456225,Cash loans,M,N,Y,Family,Working,Secondary / secondary special,Married,House / apartment,Managers,...,0.000,0.000000,0.000000,0.000000,3.000000,1,1,0,0,1


In [ ]:
##还要改

#4.one_hot_coding



#5.共线性
# Detect multicollinearity in training data and plot heatmap
corr_matrix_train = detect_multicollinearity(train_data)
# Output the correlation matrix
print("Correlation Matrix for Training Data:")
print(corr_matrix_train)

result = corr_matrix_train[corr_matrix_train > 0.9]
b= 0

# 打印行名和列名
for row in result.index:
    for col in result.columns:
        if result.at[row, col] > 0.9:  # 检查具体的值
            if row != col: 
                print(f"行名: {row}, 列名: {col}")
                b=b+1
